## Turning Wards in Geography

In our previous notebook, we normalized OpenElection ward names from the county clerk's "Reporting Units" and into an easier-to-parse and standard list of wards. Now, let's add in some Geo Data. The [LTSB Open Data Portal](http://data.ltsb.opendata.arcgis.com/) is based on ArcGIS and allows you to download data on every ward in the State as a CSV, a Shapefile, or as a KML file. To keep it simple, let's download the CSV version of the 2016 Fall wards. 

**The LTSB ArcGIS portal doesn't have 2014 ward data, so we're using 2016 ward maps to join against the 2014 OpenElections results.** This is mostly OK - a few boundaries will have changed due to annexations, and the two other big changes were two towns incorporated as villages. Strictly speaking, however, the maps don't quite line up right. We'll update this notebook to use 2016 OpenElections results in the future, and then the boundaries and results will be aligned. 

Also, this is probably as good as any place to note that the LTSB also publishes election results, and does so in as spreadsheets and shapefiles, broken down by ward. **There are no actual Statewide, ward-level results in Wisconsin. The LTSB processes the GAB/Elections Commission results to create them, just like we're doing here**. Because a given reporting unit might have multiple wards, it is impossible to say how exactly a given ward voted. The LTSB proportionally allocates votes to each ward in a reporting unit, weighted by the population of the wards. (And not, alas, by the turnout of each ward, which would at least be slightly more accurate.) If you want to know how two wards voted, you literally have to go to the Clerk's office and inspect the ballots. Also, because the LTSB has to use the Census data to weight by population, they can use the Census demographics to estimate the racial makeup of a given ward, but again, the LTSB weights by population and doesn't look at the actual Voter File to see who turned out (and Wisconsin doesn't how a voter racially identifies in the voter file anyway) these are just estimates. 

We'll start again with the preliminaries - bring in Pandas and set some blogging-friendly defaults, and then actually load the ward data.

In [1]:
import pandas as pd
import re
import json
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
# also at http://data.ltsb.opendata.arcgis.com/datasets/6497103b939d41268a48905631f84de5_0.csv
ltsbwards = pd.read_csv("http://data.ltsb.opendata.arcgis.com/datasets/6497103b939d41268a48905631f84de5_0.csv")
ltsbwards.head()

,OBJECTID,GEOID,CNTY_FIPS,COUSUBFP,WARDID,WARD_FIPS,SUPERID,SUPER_FIPS,ALDERID,ALDER_FIPS,CNTY_NAME,MCD_NAME,MCD_FIPS,CONTACT,DATE_SUB,CTV,NOTES,SCHOOLID
0,1,55001002750001,55001,275,0001,55001002750001,11,5500111,01,550010027501,ADAMS,Adams,5500100275,afaust@ncwrpc.org,7/6/2016 1:02:14 PM,C,,
1,2,55001002750002,55001,275,0002,55001002750002,12,5500112,02,550010027502,ADAMS,Adams,5500100275,afaust@ncwrpc.org,7/6/2016 1:02:14 PM,C,,
2,3,55001002750003,55001,275,0003,55001002750003,12,5500112,03,550010027503,ADAMS,Adams,5500100275,afaust@ncwrpc.org,7/6/2016 1:02:14 PM,C,,
3,4,55001002750004,55001,275,0004,55001002750004,11,5500111,04,550010027504,ADAMS,Adams,5500100275,afaust@ncwrpc.org,7/6/2016 1:02:14 PM,C,,
4,5,55001003000001,55001,300,0001,55001003000001,8,5500108,,,ADAMS,ADAMS,5500100300,afaust@ncwrpc.org,7/6/2016 1:02:14 PM,T,,


Now we need to join the LTSB Shapefile data with the GAB/OpenElex results data, to bring GeoIDs to the reporting units. The join is on (County, City) - two counties might both have the same named city (or a city might be in two counties!) so we need to join on the pair. Alas, the county names are not identitical in both data sets, and there are quirks in the city names, so we need to build up some join keys that match on both datasets. 

We'll start by modifying the LTSB data, first by creating a column to fix up the "MCD" (a 'minor civil division, in Census terms - but what we'd normally call a City, Village, or Town). We'll also create a column with the base FIPS codes for the County+MCD along with us - we'll use Ward numbers later to build out the full GEOID. (In years past, the LTSB data has already included this column but it's been dropped from the curren data)

In [3]:
def ltsb_reporting_mcd(row):
    ctv = ""
    #print(row['GEOID'])
    if row['CTV'] == 'C':
        ctv = 'CITY OF '
    elif row['CTV'] == 'T':
        ctv = 'TOWN OF '
    elif row['CTV'] == 'V':
        ctv = 'VILLAGE OF '
    else:
        raise Exception("Unexpected CTV option")
        
    return("%s%s") %(ctv, row['MCD_NAME'].upper())

def partial_fips(row):
    return("%s%s" % (row['CNTY_FIPS'], '{0:05d}'.format(int(row['COUSUBFP']))))


In [4]:
ltsbwards['JoinMcd'] = ltsbwards.apply(ltsb_reporting_mcd, axis=1)

In [5]:
ltsbwards['PARTIALFIPS'] = ltsbwards.apply(partial_fips, axis=1)

In [6]:
ltsb = ltsbwards[['CNTY_NAME', 'JoinMcd', 'PARTIALFIPS']].copy().drop_duplicates()

In [7]:
ltsb.head(5)

,CNTY_NAME,JoinMcd,PARTIALFIPS
0,ADAMS,CITY OF ADAMS,5500100275
4,ADAMS,TOWN OF ADAMS,5500100300
7,ADAMS,TOWN OF BIG FLATS,5500107300
9,ADAMS,TOWN OF COLBURN,5500116075
10,ADAMS,TOWN OF DELL PRAIRIE,5500119575


Now that we've finished processing the LTSB Ward data, let's load in our OpenElections normalized ward maps. Again, we'll have to fix some the reporting names used in that data so we can have good join keys, so we'll create two new columns, a JoinCounty and a JoinMcd.

In [8]:
openelex2014 = pd.read_csv('2014_wards_normalized.csv')
openelex2014.head()

,county,ward,Muni,ReportingUnit,mapped
0,Adams,Town Of Adams Wards 1-3,Town Of Adams,Wards 1-3,"Wards 1,2,3"
1,Adams,Town Of Big Flats Wards 1 & 2,Town Of Big Flats,Wards 1 & 2,"Wards 1,2"
2,Adams,Town Of Colburn Ward 1,Town Of Colburn,Ward 1,Ward 1
3,Adams,Town Of Dell Prairie Wards 1-3,Town Of Dell Prairie,Wards 1-3,"Wards 1,2,3"
4,Adams,Town Of Easton Wards 1 & 2,Town Of Easton,Wards 1 & 2,"Wards 1,2"


In [9]:
def map_openelex_reporting_county(cnty):
    if cnty == "La Crosse":
        return "LA_CROSSE"
    if cnty == "St. Croix":
        return "ST_CROIX"
    tokens = cnty.split()
    cntyname = "_".join(tokens)
    return cntyname.upper()

openelex2014['JoinCounty'] = openelex2014['county'].map(map_openelex_reporting_county) 

In [10]:
def map_openelex_mcd(mcd):
    mcd = mcd.upper()
    if mcd == "TOWN OF GRAND VIEW":
        return 'TOWN OF GRANDVIEW'
    elif mcd == "VILLAGE OF Windsor":
        return 'VILLAGE OF WINDSOR'
    elif mcd == "TOWN OF WINDSOR":
        return 'VILLAGE OF WINDSOR'
    elif mcd == 'VILLAGE OF FONTANA':
        return 'VILLAGE OF FONTANA-ON-GENEVA LAKE'
    elif mcd == 'TOWN OF SAINT LAWRENCE':
        return 'TOWN OF ST. LAWRENCE'
    elif mcd == 'TOWN OF LAND O-LAKES':
        return 'TOWN OF LAND O\'LAKES'
    elif mcd == 'VILLAGE OF LAVALLE':
        return 'VILLAGE OF LA VALLE'
    elif mcd == 'VILLAGE OF Maine':
        return 'VILLAGE OF MAINE'
    #elif mcd == 'TOWN OF MAINE':
    #    return 'VILLAGE OF MAINE'
    elif mcd == 'VILLAGE OF MT. STERLING':
        return 'VILLAGE OF MOUNT STERLING'
    else:
        return mcd
    
openelex2014['JoinMcd'] = openelex2014['Muni'].map(map_openelex_mcd)

There is one odditity to fix up by hand. We're using 2014 OpenElections data, however, since then the Town of Windsor and the Town of Maine in Marathon County have incorporated as villages. The Town of Windsor was easy to handle above, but it turns out there are multiple Towns in Wisconsin named 'Maine' - one in Marathon County (which has incorporated) and one in Outagamie County, which has not. So, just fix Marathon County instance by hand in the LTSB data so the 2014 OpenElections data can join against it to find the GeoIDs.

In [11]:
ltsb.loc[(ltsb['CNTY_NAME']=='MARATHON') & (ltsb['JoinMcd']=='VILLAGE OF MAINE'), 'JoinMcd'] = 'TOWN OF MAINE'

Now, we'll actually do the merge between the OpenElections data and the modified LTSB data. We want a left-join to lookup data from the LTSB side and to make sure that we cover all the OpenElections data. (We'll check to make sure there are no rows that didn't pick up the LTSB 'CNTY_NAME' data)

In [12]:
joined = pd.merge(openelex2014, ltsb, how='left', right_on=['JoinMcd', 'CNTY_NAME'], left_on=['JoinMcd', 'JoinCounty'])


In [13]:
joined.head()

,county,ward,Muni,ReportingUnit,mapped,JoinCounty,JoinMcd,CNTY_NAME,PARTIALFIPS
0,Adams,Town Of Adams Wards 1-3,Town Of Adams,Wards 1-3,"Wards 1,2,3",ADAMS,TOWN OF ADAMS,ADAMS,5500100300
1,Adams,Town Of Big Flats Wards 1 & 2,Town Of Big Flats,Wards 1 & 2,"Wards 1,2",ADAMS,TOWN OF BIG FLATS,ADAMS,5500107300
2,Adams,Town Of Colburn Ward 1,Town Of Colburn,Ward 1,Ward 1,ADAMS,TOWN OF COLBURN,ADAMS,5500116075
3,Adams,Town Of Dell Prairie Wards 1-3,Town Of Dell Prairie,Wards 1-3,"Wards 1,2,3",ADAMS,TOWN OF DELL PRAIRIE,ADAMS,5500119575
4,Adams,Town Of Easton Wards 1 & 2,Town Of Easton,Wards 1 & 2,"Wards 1,2",ADAMS,TOWN OF EASTON,ADAMS,5500122000


In [14]:
joined[joined['CNTY_NAME'].isnull()]

,county,ward,Muni,ReportingUnit,mapped,JoinCounty,JoinMcd,CNTY_NAME,PARTIALFIPS


Now, for our lookup table, we'll keep most of the OpenElections data but drop what we don't need.

In [15]:
finished = joined[['county', 'ward', 'Muni', 'ReportingUnit', 'mapped', 'PARTIALFIPS']].copy()

Finally, let's run over each row and convert the normalized OpenElections data into a set of Pipe-delimited GeoIDs, that can be used in a mapping exercise. 

In [16]:
def expandfips(row):
    #print(row'')
    #print("done")
    (junk, wardsstr) = row['mapped'].split()
    wardlist = [x for x in wardsstr.split(',')]
    fipslist = "|".join(["%s%s" % (row['PARTIALFIPS'], x.rjust(4,'0')) for x in wardlist])
    return fipslist
                     
finished['EXPANDEDGEO'] = finished.apply(expandfips, axis=1)
        

Let's just take a look at a couple of samples, including one with a funky ward ID

In [17]:
finished.head(10)

,county,ward,Muni,ReportingUnit,mapped,PARTIALFIPS,EXPANDEDGEO
0,Adams,Town Of Adams Wards 1-3,Town Of Adams,Wards 1-3,"Wards 1,2,3",5500100300,55001003000001|55001003000002|55001003000003
1,Adams,Town Of Big Flats Wards 1 & 2,Town Of Big Flats,Wards 1 & 2,"Wards 1,2",5500107300,55001073000001|55001073000002
2,Adams,Town Of Colburn Ward 1,Town Of Colburn,Ward 1,Ward 1,5500116075,55001160750001
3,Adams,Town Of Dell Prairie Wards 1-3,Town Of Dell Prairie,Wards 1-3,"Wards 1,2,3",5500119575,55001195750001|55001195750002|55001195750003
4,Adams,Town Of Easton Wards 1 & 2,Town Of Easton,Wards 1 & 2,"Wards 1,2",5500122000,55001220000001|55001220000002
5,Adams,Town Of Jackson Wards 1 & 2,Town Of Jackson,Wards 1 & 2,"Wards 1,2",5500137625,55001376250001|55001376250002
6,Adams,Town Of Leola Ward 1,Town Of Leola,Ward 1,Ward 1,5500143425,55001434250001
7,Adams,Town Of Lincoln Ward 1,Town Of Lincoln,Ward 1,Ward 1,5500144250,55001442500001
8,Adams,Town Of Monroe Ward 1,Town Of Monroe,Ward 1,Ward 1,5500153725,55001537250001
9,Adams,Town Of New Chester Ward 1-3,Town Of New Chester,Ward 1-3,"Wards 1,2,3",5500156525,55001565250001|55001565250002|55001565250003


In [18]:
finished[finished['ReportingUnit']== 'Wards 1A, 2, 4, 7']

,county,ward,Muni,ReportingUnit,mapped,PARTIALFIPS,EXPANDEDGEO
3489,Winnebago,"City Of Menasha Wards 1A, 2, 4, 7",City Of Menasha,"Wards 1A, 2, 4, 7","Wards 1A,2,4,7",5513950825,5513950825001A|55139508250002|55139508250004|5...


And now write out a lookup table to a CSV

In [19]:
finished.to_csv("2014_WI_Reporting_Units_To_GEOID.csv")